In [1]:
import pandas as pd
import urllib3 as urllib
import json
from IPython.display import JSON

In [2]:
http = urllib.PoolManager()

def getResponse(url):
    operUrl = http.request('GET', url)
    if(operUrl.status==200):
        data = operUrl.data
        jsonData = json.loads(data.decode('utf-8'))
    else:
        print("Error receiving data", operUrl.getcode())
    return jsonData

In [3]:
ltc_data = getResponse('http://www.dph.illinois.gov/sitefiles/COVIDLTC.json')
reporting_date = '%d-%02d-%02d' %(ltc_data['LastUpdateDate']['year'], ltc_data['LastUpdateDate']['month'], ltc_data['LastUpdateDate']['day'])

In [4]:
df = pd.DataFrame(ltc_data['FacilityValues'])
df['reporting_date'] = reporting_date
df['% deaths'] = df['deaths'] / df['confirmed_cases']

In [5]:
a = df.sum()
print ('Date: %s' % reporting_date)
print ('Cases: %d' % a.confirmed_cases)
print ('Deaths:  %d' % a.deaths)

Date: 2020-05-29
Cases: 17135
Deaths:  2747


In [51]:
df_county = df.groupby(by=['County']).sum()
df_county['% deaths'] = (df_county['deaths'] / df_county['confirmed_cases'])
df_county.sort_values('% deaths', ascending=False)

,confirmed_cases,deaths,% deaths
County,,,
Tazewell,6,2,0.333333
Kendall,50,16,0.320000
Jackson,36,10,0.277778
Monroe,42,11,0.261905
Madison,189,47,0.248677
LaSalle,41,9,0.219512
Jefferson,73,15,0.205479
Winnebago,147,30,0.204082
Macon,98,19,0.193878


In [58]:
import altair as alt
from vega_datasets import data
df1=df_county.sort_values(by=['deaths'], ascending=False).reset_index()
cols = ['Deaths Non LTC', 'LTC Deaths']
cols = ['LTC Deaths', 'Deaths Non LTC']


chart1 = alt.Chart(df_county.sort_values(by=['deaths'], ascending=False).reset_index()).mark_bar().encode(
    x='deaths:Q',
    y=alt.Y('County:O', sort='-x'),
    tooltip=['County', 'deaths', 'confirmed_cases', '% deaths']
)
chart2=chart1.encode(x=alt.X('% deaths', axis=alt.Axis(format='%')))
#chart2=chart1.encode(x=alt.X('% deaths'))


chart1 | chart2

alt.HConcatChart(...)

In [59]:
import altair as alt
from vega_datasets import data
df1=df_county.sort_values(by=['deaths'], ascending=False).reset_index()
cols = ['Deaths Non LTC', 'LTC Deaths']
cols = ['LTC Deaths', 'Deaths Non LTC']


chart1 = alt.Chart(df_county.sort_values(by=['deaths'], ascending=False).reset_index()).mark_bar().encode(
    x='deaths:Q',
    y=alt.Y('County:O'),
    tooltip=['County', 'deaths', 'confirmed_cases', '% deaths']
)
chart2=chart1.encode(x=alt.X('% deaths', axis=alt.Axis(format='%')))
#chart2=chart1.encode(x=alt.X('% deaths'))


chart1 | chart2 

alt.HConcatChart(...)

In [13]:
df.sort_values(by=['deaths'], ascending=False).to_csv('IL_LTC_Covid.csv')

In [14]:
df.sort_values(by=['deaths'], ascending=False)

,County,FacilityName,confirmed_cases,deaths,status,reporting_date,% deaths
505,Will,Meadowbrook Manor of Bolingbrook,188,40,Open,2020-05-29,0.212766
264,Cook,Villa at Windsor Park,153,37,Open,2020-05-29,0.241830
276,Cook,Woodbridge Nursing Pavilion,218,33,Open,2020-05-29,0.151376
184,Cook,Niles Nursing and Rehab Center,158,33,Open,2020-05-29,0.208861
200,Cook,Peterson Park Health Care,202,31,Open,2020-05-29,0.153465
...,...,...,...,...,...,...,...
197,Cook,Pearl of Rolling Meadows,3,0,Open,2020-05-29,0.000000
402,Lake,Pavilion of Waukegan,4,0,Open,2020-05-29,0.000000
405,Lake,Rolling Hills Manor,8,0,Open,2020-05-29,0.000000
406,Lake,Rolling Hills Place,6,0,Open,2020-05-29,0.000000
